In [73]:
import nibabel as nib
import os
import sys
import subprocess
import glob

In [2]:
f = "/Users/brac4g/Desktop/reg4D/test.file"

In [3]:
# touch(f)

In [4]:
# touch_empty(f)

In [23]:
class File():
    '''
    File class doc-string
    
    Attributes:
        filename (string): Filename with path to file. File does not need to exist.
    '''
    
    def __init__(self,filename):
        '''
        Return filename string
        '''
        self.filename = filename
        return None
        
    def touch(filename):
        '''
        Analagous to UNIX's touch command. Creates an empty file.

        Arguments:
            filename (string): Filename (with path)

        Returns:
            filename (string): Path to file. This now file exists and can now be used as inputs to other functions.
        '''

        if not os.path.exists(filename):
            with open(filename,"w"): pass
            filename = os.path.abspath(filename)
        else:
            print(f"file: {filename} already exists. File will not be overwritten.")

        return filename
    
    def make_filename_path(filename):
        '''
        Analagous to UNIX's touch command. Creates an empty file, and then removes it - preserving the
        absolute path to the file.

        Arguments:
            filename (string): Filename (with path)

        Returns:
            filename (string): Path to file. This now file exists and can now be used as inputs to other functions.
        '''

        if not os.path.exists(filename):
            with open(filename,"w"): pass
            filename = os.path.abspath(filename)
            os.remove(filename)
        else:
            print(f"file: {filename} already exists. File will not be overwritten.")

        return filename

In [15]:
t = File(f)

In [16]:
t

In [18]:
t.touch

<bound method File.touch of <__main__.File object at 0x7ff0d8cd9978>>

In [12]:
t = File.make_filename_path(f)

In [14]:
t

'/Users/brac4g/Desktop/reg4D/test.file'

In [55]:
class Command():
    '''
    Creates a command and an empty command list for UNIX command line programs/applications
    
    Attributes:
        command: Command to be performed on the command line
    '''
    
    def __init__(self):
        '''
        Init doc-string for Command class.
        '''
        pass; return None
    
    def init_cmd(self,command):
        '''
        Init command function for initializing commands to be used on UNIX command line.
        
        Arguments:
            command (string): Command to be used. Note: command used must be in system path
        
        Returns:
            cmd_list (list): Mutable list that can be appended to.
        '''
        self.command = command
        self.cmd_list = [f"{self.command}"]
        return self.cmd_list

In [158]:
def split_4D(nii_4d,out_prefix,dim="-t"):
    '''
    Wrapper function for FSL's fslsplit. Splits 4D nifti timeseries data and returns a list of 
    the split timeseries files.
    
    Arugments:
        nii_4d (string): Input 4D file to be separated
        out_prefix (string): Output file prefix (no '.nii.gz', preferably to separate directory)
        dim (string): Dimension to separate along [default: '-t']
        
    Returns:
        nii_list (list): Sorted list of the zeropadded filenames of the split 4D timeseries.
    '''
    
    # Check dimension argument
    if dim != "-t" and dim != "-x" and dim != "-y" and dim != "-z":
        print("Unrecognized option for dimension. Using default of time.")
        dim = "-t"
        
    
    # Initialize command
    split = Command().init_cmd("fslsplit")
    
    # Add input/output arguments
    split.append(nii_4d)
    split.append(out_prefix)
    split.append(dim)
    
    # Perform/Execute command
    subprocess.call(split)
    
    # Create list of files
    nii_list = sorted(glob.glob(out_prefix + "*.nii*"))
    
    return nii_list

In [81]:
n = split_4D("/Users/brac4g/Desktop/reg4D/test_data/filtered_func_data.nii.gz",
        "/Users/brac4g/Desktop/reg4D/test1/split",
        dim="-t")

In [156]:
def merge_4D(nii_list,out_prefix,dim="-tr",tr=2.000):
    '''
    Wrapper function for FSL's fslmerge. Merges a list of 3D nifti files and returns a merged file in some
    arbitrary dimension (t (time, with TR), x, y, or, z).
    
    Arugments:
        nii_list (list): List of the zeropadded filenames of the split 4D timeseries.
        out_prefix (string): Output file prefix (no '.nii.gz')
        dim (string): Dimension to separate along [default: '-tr']
        tr (float): Repetition Time (TR, in sec.) [default: 2.000]
        
    Returns:
        out_file (string): Merged output nifti file.
    '''
    
    # Check dimension argument
    if dim != "-tr" and dim != "-t" and dim != "-x" and dim != "-y" and dim != "-z":
        print("Unrecognized option for dimension. Using default of time with defined TR.")
        dim = "-tr"
        
    # Create empty file and path to retrieve
    out_prefix = File.make_filename_path(out_prefix)
    
    # Initialize command
    merge = Command().init_cmd("fslmerge")
    
    # Add input/output arguments
    merge.append(dim)
    merge.append(out_prefix)   
    merge.extend(nii_list)
    
    if dim == "-tr":
        merge.append(str(tr))
    
    # Perform/Execute command
    subprocess.call(merge)
    
    out_file = out_prefix + ".nii.gz"
    
    return out_file

In [157]:
merge_4D(nii_list,out_prefix,tr=3.000)

'/Users/brac4g/Desktop/reg4D/test_data/out_test_merge.nii.gz'

In [ ]:
def apply_xfm_4D(nii_file,out_prefix,ref_vol,warp="",warp_app="relative",premat="",postmat="",
                mask="",interp="",padding_size="",use_qform=False,data_type="",super_sampling=False,
                super_level="",verbose=False):
    '''
    Wrapper function for FSL's applywarp. Applies linear and non-linear transforms (xfms) in a step manner.
    
    Arugments:
        nii_file (string): Input nifti file
        out_prefix (string): Output file (no '.nii.gz')
        ref_vol (string): Reference volume
        warp (string, optional): Warp file to reference
        warp_app (string, optional): Warp field treatment. Valid options are: "relative" (default), and "absolute"
        premat (string, optional): Pre-transform linear transformation matrix
        postmat (string, optional): Post-transform linear transformation matrix
        mask (string, optional): Mask, in reference space
        interp (string, optional): Interpolation method, options include: "nn","trilinear","sinc","spline"
        padding_size (int, optional): Extrapolates outside original volume by n voxels
        use_qform (bool, optional): Use s/qforms of ref_vol and nii_file images
        data_type (string,optional): Force output data type ["char" "short" "int" "float" "double"].
        super_sampling (bool, optional): Intermediary supersampling of output [default: False]
        super_level (int, optional): Level of intermediary supersampling, a for 'automatic' or integer level. [default: 2]
        verbose (bool, optional): Enable verbose output [default: False]
        
    Returns:
        out_file (string): Output file with the applied transform(s).
    '''
    
    # Create empty file and path to retrieve
    out_prefix = File.make_filename_path(out_prefix)
    
    # Initialize command
    xfm = Command().init_cmd("applywarp")
    
    # Add required input/output arguments
    xfm.append("--in="); xfm.append(nii_file)
    xfm.append("--out="); xfm.append(out_prefix)
    xfm.append("--ref="); xfm.append(ref_vol)
    
    # Add optional input arguments
    if warp:
        xfm.append("--warp="); xfm.append(warp)
        
    if warp_app == ""
    